In [1]:
import requests
import time

In [2]:
class Transaction():
    def __init__(self, transhash, sender, receiver, value, timestamp):
        self.transhash = transhash
        self.sender = sender
        self.receiver = receiver
        self.value = value
        self.timestamp = timestamp
    
    def toString(self):
        return "{transhash: " + self.transhash + ", value: " + str(self.value) + ", timestamp: " + str(self.timestamp) + ", sender: " + ', '.join(str(e) for e in self.sender) + ", receiver: " + ', '.join(str(e) for e in self.receiver) + "}"

In [3]:
class OnchainApi():
    def __init__(self):
        self.api_key = "WEXGR6A7CZHGMMH2ZUGCZM9WY1NZUWX4VM"

    def getRequest(self, url):
        r = requests.get(url = url)
        return r.json()
        
    def getAllTransactionsByBlock(self, blockIds):
        # get transaction hash list
        trans = []
        for block in blockIds:
            block1 = self.getRequest("https://api.blockcypher.com/v1/eth/main/blocks/" + block)
            block1_trans_num = max(block1["n_tx"], 500)
            time.sleep(0.5)
            block2 = self.getRequest("https://api.blockcypher.com/v1/eth/main/blocks/" + block + "?limit=" + str(block1_trans_num))
            time.sleep(0.5)
            trans.extend(block2["txids"])
    
        results = []
        for tran in trans:
            temp = self.getTransactionByHash(tran)
            print(temp.toString())
            results.append(temp)
            time.sleep(0.4)
            
        return results
    
    # return transaction object by transaction hash
    def getTransactionByHash(self, transactionHash):
        transactions = self.getRequest("https://api.blockcypher.com/v1/eth/main/txs/" + transactionHash)
        print(transactions)
        transction = Transaction(transactionHash, transactions["inputs"][0]["addresses"], transactions["outputs"][0]["addresses"], transactions["total"]*10**(-18), transactions["confirmed"])
        
        return transction
    
    # get block number by timestamp
    def getBlockByTime(self, timestamp):
        block_result = self.getRequest("https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp=" + str(timestamp) + "&closest=before&apikey=" + self.api_key)
        return block_result["result"]
    
    def getPrice(self):
        price = self.getRequest("https://api.etherscan.io/api?module=stats&action=ethprice&apikey=" + self.api_key)
        return price["result"]["ethusd"]

In [5]:
'''
# how to use this file
api = OnchainApi()
# get all transaction details given block ID
# input: list containing block IDs
# output: list of Transaction objects 
api.getAllTransactionsByBlock(["12702435"])
# get information by transaction hash
# input: transaction hash
# ouput: Transaction object
api.getTransactionByHash("4a5cd5198911081f8dae721bdf80284f3b9fc707753318cb8089d469fea1aebe")
# get block number by timestamp
api.getBlockByTime(1578638524)
# get latest price
api.getPrice()
'''

'\n# how to use this file\napi = OnchainApi()\n# get all transaction details given block ID\n# input: list containing block IDs\n# output: list of Transaction objects \napi.getAllTransactionsByBlock(["12702435"])\n# get information by transaction hash\n# input: transaction hash\n# ouput: Transaction object\napi.getTransactionByHash("4a5cd5198911081f8dae721bdf80284f3b9fc707753318cb8089d469fea1aebe")\n# get block number by timestamp\napi.getBlockByTime(1578638524)\n# get latest price\napi.getPrice()\n'